In [1]:
from hgcal_dev.metrics.instance import match_instances, PanopticQuality
import plotly.express as px
import plotly
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from hdbscan import HDBSCAN
from sklearn.manifold import TSNE
import random

In [2]:
pred_path = Path('/home/alexjschuy/predictions/19hruike')
train_pred_path = pred_path / 'train'
val_pred_path = pred_path / 'val'

In [9]:
train_events = [np.load(f) for f in list(train_pred_path.glob('*.npz'))[:5]]
val_events = [np.load(f) for f in list(val_pred_path.glob('*.npz'))[:5]]

In [10]:
train_input_sne_figs = []
for train_event in tqdm(train_events):
    train_input_tsne = TSNE().fit_transform(train_event['features'])
    fig = px.scatter(x=train_input_tsne[:, 0], y=train_input_tsne[:, 1], color=train_event['labels'].astype('str'))
    train_input_sne_figs.append(fig)

100%|██████████| 5/5 [02:28<00:00, 29.64s/it]


In [15]:
train_input_sne_figs[1].show()

In [12]:
train_pred_sne_figs = []
for train_event in tqdm(train_events):
    train_pred_tsne = TSNE().fit_transform(train_event['prediction'])
    fig = px.scatter(x=train_pred_tsne[:, 0], y=train_pred_tsne[:, 1], color=train_event['labels'].astype('str'))
    train_pred_sne_figs.append(fig)

100%|██████████| 5/5 [02:48<00:00, 33.66s/it]


In [14]:
train_pred_sne_figs[1]

In [20]:
from hgcal_dev.training.criterion import CentroidInstanceLoss
import torch

In [21]:
criterion = CentroidInstanceLoss()

In [26]:
losses = []
for train_event_path in train_pred_path.glob('*.npz'):
    train_event = np.load(train_event_path)
    loss = criterion(torch.tensor(train_event['prediction']), torch.tensor(train_event['labels']))
    losses.append(loss)

In [28]:
losses = np.array(losses)

In [38]:
losses.mean()

3.6484742

In [32]:
losses.argmin()

6861

In [39]:
train_event = np.load(list(train_pred_path.glob('*.npz'))[losses.argmin()])

In [42]:
train_input_tsne = TSNE().fit_transform(train_event['features'])
fig = px.scatter(x=train_input_tsne[:, 0], y=train_input_tsne[:, 1], color=train_event['labels'].astype('str'))
fig.show()

In [41]:
train_pred_tsne = TSNE().fit_transform(train_event['prediction'])
fig = px.scatter(x=train_pred_tsne[:, 0], y=train_pred_tsne[:, 1], color=train_event['labels'].astype('str'))
fig.show()

In [10]:
val_input_sne_figs = []
for val_event in val_events:
    val_input_tsne = TSNE().fit_transform(val_event['features'])
    fig = px.scatter(x=val_input_tsne[:, 0], y=val_input_tsne[:, 1], color=val_event['labels'].astype('str'))
    val_input_sne_figs.append(fig)

In [11]:
val_pred_sne_figs = []
for val_event in val_events:
    val_pred_tsne = TSNE().fit_transform(val_event['prediction'])
    fig = px.scatter(x=val_pred_tsne[:, 0], y=val_pred_tsne[:, 1], color=val_event['labels'].astype('str'))
    val_pred_sne_figs.append(fig)

In [40]:
train_event = train_events[0]
clusterer = HDBSCAN(min_cluster_size=2)
clusterer.fit(train_event['prediction'])
pred_labels = clusterer.labels_
PQ = PanopticQuality(num_classes=2, min_points=2)

HDBSCAN(min_cluster_size=2)

In [42]:
pred_labels = clusterer.labels_

In [43]:
truth_to_pred, pred_to_truth, ious = match_instances(events[0]['labels'], pred_labels)

In [44]:
ious.mean()

0.10160459411696017

In [45]:
clusterer_original = HDBSCAN(min_cluster_size=2)
clusterer_original.fit(events[0]['inputs'])
pred_labels = clusterer_original.labels_
truth_to_pred, pred_to_truth, ious = match_instances(events[0]['labels'], pred_labels)
print(ious.mean())

0.0273651382085117
